In [14]:

####################################################################################################
####################################################################################################

import sys
import os.path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

import keras
import keras.utils.vis_utils
from keras import backend as K
from keras import layers
from keras.layers import Dense
from keras.models import load_model
from keras.models import Sequential
from keras.utils import plot_model

from helpers import *

sys.path.append(os.path.abspath("../"))
from pax_utils import s1s2_utils

v_drift = 1.44e-4


In [15]:

####################################################################################################
####################################################################################################

nEventsTrain = 100000
file_all     = "../data_lfs/merged_all_200k_1803261941.pkl"

df_all = pd.read_pickle(file_all)
df_all = df_all[df_all['intr_count'] == 1].reset_index(drop=True)
df_all = df_all[nEventsTrain:][:]

#df_all['intr_depth'] = v_drift*df_all['intr_drift_time'] 
#df_all['intr_depth'] = -1*v_drift*df_all['intr_drift_time'] 
df_all['depth']      *= -1


####################################################################################################
####################################################################################################

df_train_truth = df_all[:][['depth']]
#df_train_input = df_all[:][s1s2_utils.getS2integralsDataFrameColumns()]
df_train_input = df_all[:]['intr_drift_time']
#df_train_input = df_all[:][['intr_depth'] + s1s2_utils.getS2integralsDataFrameColumns()]


####################################################################################################
####################################################################################################

checkTrainingData(df_train_input, df_train_truth)




Input shape: (20967,)
Truth shape: (20967, 1)

Input DataFrame: 


100000    518770.0
100001    317870.0
100002    470950.0
100003    574080.0
100004    530950.0
Name: intr_drift_time, dtype: float64


Truth DataFrame:


,depth
100000,-74.424872
100001,-45.412076
100002,-67.404493
100003,-82.343068
100004,-76.135639


In [16]:

####################################################################################################
####################################################################################################

model_name     = "model_barge_z_elu"
folder         = "models" + "/"    
model_name_h5  = folder + model_name + ".h5"
model_name_png = folder + model_name + ".png"



In [17]:

####################################################################################################
####################################################################################################

model       = load_model(model_name_h5)
arr_xy_pred = model.predict(df_train_input)

print("\nLoaded Model: " + model_name_h5)
print("Predicted\n")




Loaded Model: models/model_barge_z_elu.h5
Predicted



In [18]:

####################################################################################################
####################################################################################################

arr_z_true = df_train_truth[:]['depth'].as_matrix()
arr_z_pred = arr_xy_pred[:, 0]
arr_dz     = arr_z_true - arr_z_pred



In [19]:

####################################################################################################
####################################################################################################

file_out         = "./predictions/z_DeltaT.pkl"

cols_out         = ['event_number', 'depth', 'intr_z']

df_out           = df_all[:][cols_out]
df_out['z_pred'] = arr_z_pred


df_out.to_pickle(file_out)


####################################################################################################
####################################################################################################

df_test = pd.read_pickle(file_out)

display(df_test[0:5][:])
print()



,event_number,depth,intr_z,z_pred
100000,165461,-74.424872,-74.45808,-36.792614
100001,165462,-45.412076,-45.52848,-22.554335
100002,165463,-67.404493,-67.57200,-33.409801
100003,165468,-82.343068,-82.42272,-40.723278
100004,165469,-76.135639,-76.21200,-37.657848
